<a href="https://colab.research.google.com/github/medulka/LLMs/blob/main/RIG_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RIG**

retrieval interleaved generation
A postprocessing method.

Uses model Gemma2, finetuned and named as DataGemma2. The model replace a generated number with an information from Data Commons. Double-checking the answer against a trusted source.
The DataGemma model generates a response iuncluding a natural language query for Data Commons' existing natural language interface, specially designed to retrieve relevant data.
The final response is presented to the user, including a link to the source data and metadata in Data Commons for verification.

*modified after:* https://colab.research.google.com/github/datacommonsorg/llm-tools/blob/main/notebooks/datagemma_rig.ipynb

In [ ]:
!pip install -q git+https://github.com/datacommonsorg/llm-tools
!pip install -q bitsandbytes accelerate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.1 MB/s eta 0:00:00


In [ ]:
import torch

import data_gemma as dg

from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# initialization of the Data Commons API client
DC_API_KEY = userdata.get('DC_API_KEY')
dc = dg.DataCommons(api_key=DC_API_KEY)

# downloading finetuned GEmma2 model from HuggingFace
HF_TOKEN = userdata.get('HF_TOKEN')

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16)

model_name = 'google/datagemma-rig-27b-it'
tokenizer = AutoTokenizer.from_pretrained(model_name, token = HF_TOKEN)
datagemma_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                       device_map = "auto",
                                                       quantization_config = nf4_config,
                                                       torch_dtype = torch.bfloat16,
                                                       token = HF_TOKEN)

# Build the LM Model stub to use in RIG flow
datagemma_model_wrapper = dg.HFBasic(datagemma_model, tokenizer)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/850 [00:00<?, ?B/s]

...

model-00010-of-00012.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/680M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

pose a question

In [ ]:
QUERY = "What is the global demographic prediction for the year 2050?"

running RIG

In [ ]:
# printing out the answer of the model
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
    formatted_prompt = "<font size='+1' color='blue'><blockquote>" + prompt + "</blockquote></font>"
    text = text.replace('•', '  *')
    text = textwrap.indent(text, '>', predicate= lambda _: True)
    formatted_text = "<font size='+1' color='blue'>\n\n" + text + "\n</font>"
    return Markdown(formatted_prompt + formatted_text)

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '>', predicate= lambda _: True))

ans = dg.RIGFlow(llm=datagemma_model_wrapper, data_fetcher=dc, verbose=False).query(query=QUERY)
Markdown(textwrap.indent(ans.answer(), '> ', predicate=lambda _: True))

display_chat(QUERY, ans.answer())

... calling HF Pipeline API "What is the global demographic prediction for the..."


... calling DC with "what is the predicted global population in 2050?"


<font size='+1' color='blue'><blockquote>What is the global demographic prediction for the year 2050?</blockquote></font><font size='+1' color='blue'>

>
>
>The global demographic prediction for 2050 is a complex and multifaceted issue.
>
>**Here's a breakdown:**
>
>* **Population Growth:** The world's population is expected to continue growing, reaching approximately [__DC__("what is the predicted global population in 2050?") --> "8 billion"] by 2050. This is largely due to increasing life expectancy and a still-high fertility rate in many parts of the world.
>* **Aging Population:** While the global population grows, it's also aging. This means the proportion of older people is increasing, which can put pressure on healthcare and social security systems.
>* **Regional Variations:**
>
>    * **Sub-Saharan Africa:** This region has the highest population growth rate, estimated at around [__DC__("what is the predicted population growth rate in Sub-Saharan Africa?") --> "3%"].
>    * **Developed Countries:** Many developed countries have growth rates closer to [__DC__("what is the predicted population growth rate in developed countries?") --> "0%"] or slightly above.
>
>* **Factors Influencing Growth:**
>
>    * **High Fertility Rates:** Many African countries have high birth rates, often due to cultural norms, lack of access to family planning, and high infant mortality rates.
>    * **Declining Mortality Rates:** Improvements in healthcare, sanitation, and nutrition have led to a decrease in death rates, particularly among children.
>    * **Young Population Structure:**
>
>A large proportion of the population is in their reproductive years, contributing to the growing number of young people.
>
>**Challenges of Global Population Growth:**
>
>* **Strain on Resources:** Increased population puts pressure on natural resources like water, food, and energy.
>* **Unemployment and Poverty:** Rapid population growth can lead to high unemployment rates and exacerbate poverty.
>* **Environmental Degradation:** Population growth contributes to deforestation, pollution, and other environmental problems.
>* **Overcrowding and Urbanization:** Rapid urbanization and overcrowding in cities can lead to challenges in providing adequate housing, education, and healthcare.
>
>**Positive Aspects of Global Population Growth:**
>
>* **Diverse Workforce:** A younger, growing population can lead to a more diverse and dynamic workforce.
>* **Innovation and Entrepreneurship:**
>
>Population growth can foster innovation and entrepreneurship as countries seek to adapt to the growing population.
>
>* **Improved Living Standards:**
>
>Access to education, healthcare, and other services can lead to improvements in living standards, especially for children.
>
>**Overall, while global population is predicted to continue growing, it's important to note that this is a general overview, and specific figures can vary depending on the source and methodology used.**
>
>For the most up-to-date and detailed information, it's best to consult sources like the United Nations Population Division, the World Bank, and other reputable organizations working on demographics.

</font>